<a href="https://colab.research.google.com/github/KhaSab5292/DS-Unit-2-Applied-Modeling/blob/master/Copy_of_assignment_applied_modeling_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Is your problem regression or classification?
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency > 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%%capture
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


DATA_PATH = '/content/drive/My Drive/Lambda/Unit 2 DS9/Unit 2 Build/Police_Department_Incidents_-_Previous_Year__2016_.csv'
!pip install category_encoders==2.*

In [5]:
df = pd.read_csv(DATA_PATH)
print(df.shape)
df.head()

(150500, 13)


,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000


# **Choose your target. Which column in your tabular dataset will you predict?**

In [6]:
df.describe()

,IncidntNum,X,Y,PdId
count,1.505000e+05,150500.000000,150500.000000,1.505000e+05
mean,1.616440e+08,-122.423599,37.768921,1.616440e+13
std,5.535976e+06,0.026210,0.023637,5.535976e+11
min,1.135121e+07,-122.513642,37.707922,1.135121e+12
25%,1.603283e+08,-122.434036,37.756486,1.603283e+13
50%,1.606541e+08,-122.416903,37.775421,1.606541e+13
75%,1.609764e+08,-122.406605,37.785063,1.609764e+13
max,9.910090e+08,-122.365565,37.819975,9.910090e+13


In [7]:
df.describe(exclude = 'number')

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,Location
count,150500,150500,150500,150500,150500,150499,150500,150500,150500
unique,39,726,7,366,1439,10,14,16130,19386
top,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Friday,01/01/2016 12:00:00 AM,12:00,SOUTHERN,NONE,800 Block of BRYANT ST,"(37.775420706711, -122.403404791479)"
freq,40409,17741,23371,558,3842,28445,107780,3561,3536


In [0]:
target = 'Resolution'

# **Is your problem regression or classification?**

In [9]:
df[target].value_counts(normalize = True)

NONE                                      0.716146
ARREST, BOOKED                            0.261900
UNFOUNDED                                 0.010684
JUVENILE BOOKED                           0.007017
EXCEPTIONAL CLEARANCE                     0.002465
ARREST, CITED                             0.000957
CLEARED-CONTACT JUVENILE FOR MORE INFO    0.000385
NOT PROSECUTED                            0.000146
LOCATED                                   0.000133
PSYCHOPATHIC CASE                         0.000113
JUVENILE CITED                            0.000020
COMPLAINANT REFUSES TO PROSECUTE          0.000013
JUVENILE DIVERTED                         0.000013
PROSECUTED BY OUTSIDE AGENCY              0.000007
Name: Resolution, dtype: float64

Classification

# **How many classes? Are the classes imbalanced?**

In [10]:
print(df[target].nunique(), 'unique values')

14 unique values


# **Choose which observations you will use to train, validate, and test your model.**

In [11]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, train_size = 0.8, test_size = 0.2,  
                               random_state = 42)
df.shape, train.shape, test.shape

((150500, 13), (120400, 13), (30100, 13))

In [12]:
train, val = train_test_split(train, random_state = 42)
train.shape, val.shape, test.shape

((90300, 13), (30100, 13), (30100, 13))

# **Choose your evaluation metric(s).**

accuracy score

In [0]:
from sklearn.metrics import accuracy_score

Classification: Is your majority class frequency > 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?

precision, recall, and roc auc

# **Begin to clean and explore your data.**

In [14]:
df['Category'].value_counts(normalize = True)

LARCENY/THEFT                  0.268498
OTHER OFFENSES                 0.130226
NON-CRIMINAL                   0.118711
ASSAULT                        0.090213
VANDALISM                      0.057070
VEHICLE THEFT                  0.042651
WARRANTS                       0.039296
BURGLARY                       0.038551
SUSPICIOUS OCC                 0.038419
MISSING PERSON                 0.028824
DRUG/NARCOTIC                  0.028193
ROBBERY                        0.021920
FRAUD                          0.017508
SECONDARY CODES                0.012233
TRESPASS                       0.012040
WEAPON LAWS                    0.011017
SEX OFFENSES, FORCIBLE         0.006246
STOLEN PROPERTY                0.005860
RECOVERED VEHICLE              0.004890
DISORDERLY CONDUCT             0.004372
PROSTITUTION                   0.004259
FORGERY/COUNTERFEITING         0.004113
DRUNKENNESS                    0.003090
DRIVING UNDER THE INFLUENCE    0.002512
ARSON                          0.001900


In [15]:
df['DayOfWeek'].value_counts(normalize = True)

Friday       0.155289
Saturday     0.147322
Thursday     0.142159
Wednesday    0.141741
Tuesday      0.141143
Monday       0.138093
Sunday       0.134252
Name: DayOfWeek, dtype: float64

In [16]:
df['PdDistrict'].value_counts(normalize = True)

SOUTHERN      0.189005
NORTHERN      0.133556
MISSION       0.129589
CENTRAL       0.117383
BAYVIEW       0.095037
INGLESIDE     0.077037
TARAVAL       0.075250
TENDERLOIN    0.066060
RICHMOND      0.059283
PARK          0.057801
Name: PdDistrict, dtype: float64

# **Begin to choose which features, if any, to exclude. Would some features "leak" future information?**

In [0]:
exluded_columns = ['IncidntNum', 'Descript', 'Date', 'Time', 'PdId', 'Address', 'Location']